In [1]:
cd ../

/Users/arbaaz/Downloads/mabe/mabe_code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd


train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [4]:
from data_utils import get_col_names


cols = get_col_names("_color")
cols

['mouse1_color', 'mouse2_color', 'mouse3_color', 'mouse4_color']

In [5]:
[train_metadata[col].isna().any() for col in cols]

[np.False_, np.False_, np.True_, np.True_]

In [6]:
[train_metadata[col].head() for col in cols]

[0    white
 1    white
 2    white
 3    white
 4    white
 Name: mouse1_color, dtype: object,
 0    white
 1    white
 2    white
 3    white
 4    white
 Name: mouse2_color, dtype: object,
 0    white
 1    white
 2    white
 3    white
 4    white
 Name: mouse3_color, dtype: object,
 0    white
 1    white
 2    white
 3      NaN
 4      NaN
 Name: mouse4_color, dtype: object]

In [7]:
# unique color of mouse global
for col in cols:
    print(col)
    print(train_metadata[col].unique())
    print()

mouse1_color
['white' 'black' 'brown' 'black and tan']

mouse2_color
['white' 'black' 'brown' 'black and tan']

mouse3_color
['white' nan 'black and tan' 'black']

mouse4_color
['white' nan]



In [8]:
from collections import defaultdict


# unique ages of each mouse by lab

color_by_lab = defaultdict(lambda: defaultdict(set))

for lab_id, grp in train_metadata.groupby("lab_id"):
    for col in cols:
        color_by_lab[lab_id][col].update(grp[col].dropna().unique())
        
nested_dict = {
    lab: {k: ', '.join(v) if v else '' for k, v in inner.items()}
    for lab, inner in color_by_lab.items()
}

# create dataframe
df = pd.DataFrame.from_dict(nested_dict, orient='index')

# display
df

,mouse1_color,mouse2_color,mouse3_color,mouse4_color
AdaptableSnail,white,white,white,white
BoisterousParrot,black,black,,
CRIM13,black,white,,
CalMS21_supplemental,black,white,,
CalMS21_task1,black,white,,
CalMS21_task2,black,white,,
CautiousGiraffe,black,black,,
DeliriousFly,black,white,,
ElegantMink,"black, brown","black, brown",,
GroovyShrew,black,black,,


In [ ]:
import pandas as pd
from collections import defaultdict

# Step 1: collect counts
color_counts = defaultdict(lambda: defaultdict(int))

for lab_id, grp in train_metadata.groupby("lab_id"):
    for col in cols:
        for strain, count in grp[col].value_counts().items():
            if pd.notna(strain):
                color_counts[lab_id][strain] += count

# Step 2: convert to DataFrame
df = pd.DataFrame.from_dict(
    {lab: dict(strains) for lab, strains in color_counts.items()},
    orient='index'
).fillna(0).astype(int)

df.index.name = 'lab_id'
df.columns.name = 'strain'

# Step 3: display
df

strain,white,black,brown,black and tan
lab_id,,,,
AdaptableSnail,61,0,0,0
CRIM13,21,21,0,0
CalMS21_supplemental,297,297,0,0
CalMS21_task1,101,101,0,0
CalMS21_task2,76,76,0,0
DeliriousFly,6,6,0,0
InvincibleJellyfish,33,33,0,0
JovialSwallow,52,52,0,0
LyricalHare,24,24,0,0


In [10]:
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [11]:
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
